<a href="https://colab.research.google.com/github/alirezakhaleghianaghizi/MLC/blob/main/AI%D9%80Course%D9%80Challenge%D9%80NLP%D9%80Phase1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div dir="rtl">
<font face="B Nazanin" size=5>
    <b>بسم الله الرحمن الرحیم</b>
</font>
</div>

<div dir="rtl">
<font face="B Nazanin" size=5>
    <b>چالش درس هوش مصنوعی - پردازش زبان طبیعی</b>
    <br>
    <b>فاز اول</b>
</font>
</div>

<div dir="rtl">
<font face="B Nazanin" size=4>
نوت بوکی که پیش روی شماست، مراحل انجام فاز اول چالش درس هوش مصنوعی در قسمت پردازش زبان طبیعی را تعیین کرده است. در این فاز، شما با استفاده مجموعه‌ای از محتوای متن صفحات وب به همراه برچسب موضوع هر یک، مدلی را توسعه می‌دهید که توانایی دسته‌بندی متن ورودی را به یکی از 16 برچسب مجموعه‌ داده داشته باشد.
<br>
<br>
<b>چند نکته مهم</b>
<ul >
<font face="B Nazanin" size=4>
<li>
می‌توانید از کتابخانه‌های با پیاده‌سازی آماده مدل‌های پایه (مانند transformers) استفاده کنید.
</li>
<li>
توجه کنید که تمام کتابخانه های مورد نیاز را نصب کنید و از نصب شدن آن ها اطمینان حاصل کرده و نام آن ها در فایل requirements ذکر کنید.
</li>
<li>
در صورت نیاز به نصب یک کتابخانه یا دسترسی به یک فایل، مراحل نصب و دانلود آن (از یک محل عمومی یا فایل ارسال شده به همراه نوت‌بوک) می‌بایست در نوت‌بوک وجود داشته باشد.
</li>
<li>
تمامی فایل‌های مرتبط به پروژه که دارای حجم کمی هستند، باید به همراه نوت‌بوک در قالب یک فایل زیپ ارسال شوند. فایل‌هایی که حجم زیادی دارند (مانند فایل ذخیره شده یک مدل که بیشتر از ۲۰۰ مگابایت باشد) را در یک محل عمومی مانند گوگل درایو آپلود کرده و لینک دسترسی به آن را در نوت بوک قرار دهید.
</li>
</font>
</ul>
</font>
</div>

<div dir="rtl">
<font face="B Nazanin" size=4>
<b>بارگذاری و پیش‌پردازش داده</b>
<br>
در ابتدا می‌بایست داده‌ها را از فایل با نام dataset_phase1 بارگذاری کنید. سپس برای اینکه بتوانید مدل مناسبی را آموزش دهید،  داده‌ها را پیش پردازش کنید. این قسمت می‌تواند شامل موارد متنوعی باشد؛ از جمله حذف عبارت‌های اضافی موجود در انتهای متن‌ها، حذف یا جداسازی علائم نگارشی، توکن‌بندی کلمات، جایگزینی مواردی مثل اعداد، تاریخ‌ها و آدرس‌‌های الکترونیکی با توکن‌های خاص، حذف فاصله‌های اضافه و ... . برای این منظور، می‌توانید از کتاب‌خانه‌هایی مانند hazm هم استفاده کنید.
سپس برای اینکه بتوانید ارزیابی مناسبی از مدل خود داشته باشید، داده‌ها را به دو قسمت آموزش  (train) و ارزیابی  (validation) تقسیم کنید.
</font>
</div>

In [32]:
import csv
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from google.colab import drive
import string
import re
from nltk.stem.porter import PorterStemmer
import nltk
!pip install hazm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from hazm import *

# data must be uploded to the drive
drive.mount('/content/drive')
filename='/content/drive/MyDrive/ml/phase_1_train.csv'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
df = pd.read_csv(filename)
df.head()

,text,class_id,class_name
0,تداوم فاز انتظاری در بازار خودرو/دنا پلاس امرو...,21,خودرو
1,موتورسیکلت آلمانی با پیشرانه وی 6 | فیلم - نیو...,21,خودرو
2,کدام خودروها در مهر 1401 امکان واردات دارند؟ -...,21,خودرو
3,\r\n خودرو از 0 تا 12762 میلیون تومان\r...,21,خودرو
4,آبروریزی جنجالی سایپا در نمایشگاه خودرو گزارش ...,21,خودرو


In [3]:
X = df.drop('class_id', axis=1)
#X = df.drop('class_name', axis=1)
y_id = df['class_id']
y_name = df['class_name']


In [68]:
string.punctuation
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree
X['clean_msg']= X['text'].apply(lambda x:remove_punctuation(x))

def remove_numbers(text):
    result = re.sub(r'\d+', '', text)
    return result
X['number_removed']=X['clean_msg'].apply(lambda x: remove_numbers(x))

def tokenization(text):
    tokens = word_tokenize(text)
    return tokens
#applying function to the column
X['msg_tokenied']= X['number_removed'].apply(lambda x: tokenization(x))
#importing the Stemming function from nltk library

lemmatizer = Lemmatizer()

#defining a function for lemmatizer
def mylemmatizer(text):

    lemma_text = [lemmatizer.lemmatize(word) for word in text]
    return lemma_text
X['msg_lemmatized']=X['msg_tokenied'].apply(lambda x: mylemmatizer(x))
X.head()




,text,class_name,clean_msg,number_removed,msg_tokenied,msg_lemmatized
0,تداوم فاز انتظاری در بازار خودرو/دنا پلاس امرو...,خودرو,تداوم فاز انتظاری در بازار خودرودنا پلاس امروز...,تداوم فاز انتظاری در بازار خودرودنا پلاس امروز...,"[تداوم, فاز, انتظاری, در, بازار, خودرودنا, پلا...","[تداوم, فاز, انتظار, در, بازار, خودرودنا, پلاس..."
1,موتورسیکلت آلمانی با پیشرانه وی 6 | فیلم - نیو...,خودرو,موتورسیکلت آلمانی با پیشرانه وی 6 فیلم نیوزی...,موتورسیکلت آلمانی با پیشرانه وی فیلم نیوزین...,"[موتورسیکلت, آلمانی, با, پیشرانه, وی, فیلم, نی...","[موتورسیکلت, آل, با, پیشرانه, وی, فیلم, نیوزین..."
2,کدام خودروها در مهر 1401 امکان واردات دارند؟ -...,خودرو,کدام خودروها در مهر 1401 امکان واردات دارند؟ ...,کدام خودروها در مهر امکان واردات دارند؟ مجله...,"[کدام, خودروها, در, مهر, امکان, واردات, دارند,...","[کدام, خودرو, در, مهر, امکان, واردات, داشت#دار..."
3,\r\n خودرو از 0 تا 12762 میلیون تومان\r...,خودرو,\r\n خودرو از 0 تا 12762 میلیون تومان\r...,\r\n خودرو از تا میلیون تومان\r\n ...,"[\r, خودرو, از, تا, میلیون, تومان\r, نمایشگاه,...","[\r, خودرو, از, تا, میلیون, تومان\r, نمایشگاه,..."
4,آبروریزی جنجالی سایپا در نمایشگاه خودرو گزارش ...,خودرو,آبروریزی جنجالی سایپا در نمایشگاه خودرو گزارش ...,آبروریزی جنجالی سایپا در نمایشگاه خودرو گزارش ...,"[آبروریزی, جنجالی, سایپا, در, نمایشگاه, خودرو,...","[آبروریزی, جنجال, سایپا, در, نمایشگاه, خودرو, ..."


below is not completed yet

we shoud count each word frequency of happened in each group and by those number we can model the nlp for persian text

below is code for two group of text in english for tweets

In [46]:
def create_freq(tweets):
  freq = {}
  for tweet in tweets:
    for word in tweet:
      if word in freq:
        freq[word] += 1
      else:
        freq[word] = 1
  return freq

freq_classes={}
for clas in set(y_name):
  freq_classes[clas]=create_freq(X[(X.class_name==clas)]['msg_lemmatized'].tolist())
  #for key in list(freq_classes[clas]):
   # if freq_classes[clas][key] <sum(freq_classes[clas].values())/100 :
    #  del freq_classes[clas][key]

In [47]:

#positive tweets and  negetiv tweets are two group of our text

def convert_text_to_vector(texts, freq_classes,y_name):
  vectors = np.zeros((len(texts), 16))
  for index, text in enumerate(texts):
    #process tweet return the tokenized of sentence for our data we should use X["msg_lemmatized"]
    #print(text)
    for clas in range(len(set(y_name))):
      yset=list(set(y_name))
      #print()
      for word in text:
        #print("word",word)
        if word in freq_classes[yset[clas]].keys():
         val=freq_classes[yset[clas]].get(word, 0)
         if not(val ==None) :

           vectors[index][clas] += val
    vectors[index] /= np.linalg.norm(vectors[index])
  return vectors

text_to_vectorss={}
for clas in set(y_name):
  text_to_vectorss[clas]=convert_text_to_vector(X[(X.class_name==clas)]['msg_lemmatized'].tolist(), freq_classes, y_name)



print((text_to_vectorss))



{'غذا و نوشیدنی': array([[0.24996801, 0.05124291, 0.20330537, ..., 0.36940919, 0.1750197 ,
        0.29924413],
       [0.24050032, 0.05188866, 0.19224438, ..., 0.36410799, 0.17035842,
        0.30911552],
       [0.2070496 , 0.05481086, 0.20752361, ..., 0.38229067, 0.17306035,
        0.29882318],
       ...,
       [0.21477961, 0.05004976, 0.20729149, ..., 0.38562944, 0.15756247,
        0.28804588],
       [0.25097216, 0.05291325, 0.2114881 , ..., 0.36091562, 0.19764015,
        0.31885355],
       [0.26688884, 0.05117948, 0.19558238, ..., 0.35494784, 0.18559865,
        0.3131447 ]]), 'حیوانات خانگی': array([[0.19471146, 0.0485999 , 0.20753457, 0.58250846, 0.10992226,
        0.29495669, 0.13048422, 0.05193664, 0.06620804, 0.18601575,
        0.30727847, 0.04880793, 0.2514426 , 0.39227294, 0.16306956,
        0.28116213],
       [0.20963774, 0.06010032, 0.20462262, 0.5712821 , 0.10172176,
        0.31156849, 0.12817801, 0.061635  , 0.05997829, 0.18592289,
        0.3061561 , 0.0518

In [48]:
X_train =text_to_vectorss['غذا و نوشیدنی']
Y_train=['غذا و نوشیدنی'] * len(text_to_vectorss['غذا و نوشیدنی'])

for clas in set(y_name):
  if not(clas =='غذا و نوشیدنی'):
   X_train=  np.concatenate((X_train,text_to_vectorss[clas]), axis=0)
   Y_train = np.concatenate((Y_train,[clas] * len(text_to_vectorss[clas])),axis=0)
print(len(Y_train),len(X_train))

2702 2702


In [49]:
TEST_SIZE = 0.3
X_train, X_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=TEST_SIZE)


<div dir="rtl">
<font face="B Nazanin" size=4>
<b>آموزش و ارزیابی مدل</b>
<br>
در این قسمت شما به آماده‌سازی مدل انتخابی و آموزش دسته‌بند می‌پردازید. برای این‌که بتوانید به نتایج بهتری برسید، مقادیر مختلف پارامترها و ابرپارامترهای مدل را با استفاده از داده اعتبارسنجی ارزیابی کنید و بهترین مقادیر را انتخاب کنید. در ارزیابی مدل می‌توانید از معیارهای متفاوتی مانند
      Recall ،Precision ،Accuracy ، Weighted F1-Scoreو F1-Score استفاده کنید.
پس از انتخاب پارامترهای بهینه، می‌توانید مدل نهایی خود را روی تمامی داده‌ها آموزش دهید.
</font>
</div>

In [50]:
model = LogisticRegression(max_iter=500)
model.fit(X_train,y_train)

LogisticRegression(max_iter=500)

In [56]:
y_pred = model.predict(X_test)
accuracy=0
for i in range(len(y_pred)):
  if y_pred[i]==y_test[i]:
    accuracy += 1

accuracy=accuracy/len(y_pred)
print((accuracy))

0.4562268803945746


In [72]:
dict_name_to_id={}
for clas in set(y_name):
  dict_name_to_id[clas]=list(set(df.loc[df['class_name'] == clas]['class_id']))[0]


In [73]:
print(dict_name_to_id)

{'غذا و نوشیدنی': 14, 'حیوانات خانگی': 7, 'سفر و گردشگری': 16, 'حقوق و دولت و سیاست': 10, 'کتاب و ادبیات': 13, 'تکنولوژی و کامپبوتر': 20, 'فیلم و سینما': 6, 'خانواده': 18, 'مسکن': 8, 'مذهبی': 17, 'ورزش': 12, 'علم و دانش': 9, 'خودرو': 21, 'تجارت و اقتصاد': 19, 'خانه و باغبانی': 15, 'هنر و سرگرمی': 11}


<div dir="rtl">
<font face="B Nazanin" size=4>
<b>دسترسی به مدل</b>
<br>
برای ارزیابی مدل شما توسط داده آزمایش (test)، تابعی با نام classify_text بنویسید که به عنوان ورودی دیتافریم تست را دریافت کند و در خروجی یک لیست از اعداد فیلد 'class_id' (از جنس int) متناظر با متن دیتافریم ورودی در ردیف با همان index است را با توجه به موضوع متن تولید کند( test_dataframe.iloc[i] -> results[i] ). این تابع توسط دستیاران آموزشی روی داده‌های آزمایش اجرا شده و نتایج آن برای ارزیابی کیفیت مدل شما استفاده خواهد شد.
</font>
</div>

In [76]:
def classify_text(test_dataframe):
    string.punctuation
    X = test_dataframe
    #X = df.drop('class_name', axis=1)
    y_id = df['class_id']
    y_name = df['class_name']
    X['clean_msg']= X['text'].apply(lambda x:remove_punctuation(x))
    X['number_removed']=X['clean_msg'].apply(lambda x: remove_numbers(x))
    X['msg_tokenied']= X['number_removed'].apply(lambda x: tokenization(x))
    lemmatizer = Lemmatizer()
    X['msg_lemmatized']=X['msg_tokenied'].apply(lambda x: mylemmatizer(x))
    text_to_vectorss={}
    X_train=text_to_vectorss[clas]=convert_text_to_vector(X['msg_lemmatized'].tolist(), freq_classes, y_name)
    base_label= model.predict(X_train)
    labels=[]
    for i in base_label:
      labels.append(dict_name_to_id[i])
    return labels

results = classify_text(df)

In [77]:
print(results)

[21, 10, 21, 21, 12, 21, 12, 10, 21, 21, 10, 21, 10, 11, 12, 21, 10, 10, 10, 21, 10, 21, 21, 10, 12, 10, 10, 12, 21, 21, 12, 21, 10, 21, 10, 21, 21, 21, 21, 21, 21, 10, 21, 21, 11, 21, 12, 11, 21, 10, 11, 21, 21, 21, 12, 10, 12, 10, 21, 10, 21, 12, 21, 21, 12, 12, 21, 10, 21, 11, 21, 10, 21, 12, 21, 10, 21, 12, 21, 21, 10, 21, 10, 12, 12, 12, 10, 10, 21, 12, 21, 21, 21, 10, 21, 10, 21, 21, 20, 10, 10, 10, 10, 10, 10, 21, 11, 21, 10, 12, 10, 21, 21, 21, 20, 21, 21, 10, 21, 12, 10, 12, 10, 10, 21, 21, 21, 21, 10, 21, 10, 11, 21, 12, 10, 10, 21, 10, 21, 20, 21, 21, 10, 21, 10, 21, 21, 21, 21, 12, 11, 21, 21, 21, 21, 21, 12, 10, 21, 12, 12, 21, 21, 10, 10, 21, 21, 12, 21, 10, 10, 21, 21, 21, 21, 6, 10, 21, 10, 21, 21, 10, 21, 21, 10, 21, 21, 12, 21, 10, 21, 21, 11, 11, 10, 11, 11, 11, 12, 11, 11, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 11, 11, 10, 12, 12, 11, 12, 12, 11, 11, 10, 11, 11, 11, 11, 11, 11, 12, 11, 11, 11, 10, 11, 12, 12, 11, 11, 11, 11, 6, 11, 11, 12, 10, 10, 1